## Imports and Installation

In [ ]:
!pip install torchmetrics thop

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 25.6 MB/s eta 0:00:00


In [ ]:
import torch
import torchvision
import albumentations as A
import pandas as pd
import cv2
import os
import time
import shutil
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from tqdm.auto import tqdm
from torchmetrics.detection import MeanAveragePrecision
import matplotlib.pyplot as plt
from thop import profile

In [ ]:
!unzip /content/dataset_cropped_224x224_newest.zip -d /content/dataset_cropped_224x224_newest

Archive:  /content/dataset_cropped_224x224_newest-20251128T144454Z-1-001.zip
  inflating: /content/dataset_cropped_224x224_newest/dataset_cropped_224x224_newest/train/b--50-_jpg.rf.fa7945cbc17bad7cb2a6f51a138c0b99_tile1.jpg  
  inflating: /content/dataset_cropped_224x224_newest/dataset_cropped_224x224_newest/train/125_jpg.rf.f37f515011dff4fbb0b882e824432b93_tile2.jpg  
  inflating: /content/dataset_cropped_224x224_newest/dataset_cropped_224x224_newest/train/a--32-_jpg.rf.f7a6bf6256f4a4bc417a23dd47536c7f_tile1.jpg  
  inflating: /content/dataset_cropped_224x224_newest/dataset_cropped_224x224_newest/train/a--87-_jpg.rf.e99dd21bd33d8f9c6b36c8d8b908f5f8_tile2.jpg  
  inflating: /content/dataset_cropped_224x224_newest/dataset_cropped_224x224_newest/train/c--142-_jpg.rf.fb5f460875d029f6681a6b655358776e_tile2.jpg  
  inflating: /content/dataset_cropped_224x224_newest/dataset_cropped_224x224_newest/train/156_jpg.rf.746421f245b72dbf541074bf5d373bdd_tile0.jpg  
  inflating: /content/dataset_crop

## Data preprocessing

In [ ]:
# if os.path.exists('./dataset_tiles'):
#     shutil.rmtree('./dataset_tiles')

# def crop_image_into_4tiles(img, tile_w=224, tile_h=224):
#     h, w, _ = img.shape
#     overlap_w = (tile_w * 2 - w) if w < tile_w * 2 else 0
#     overlap_h = (tile_h * 2 - h) if h < tile_h * 2 else 0

#     stride_w = tile_w - overlap_w
#     stride_h = tile_h - overlap_h

#     tiles = []
#     positions = []
#     tiles.append(img[0:tile_h, 0:tile_w])
#     positions.append((0, 0))
#     tiles.append(img[0:tile_h, w-tile_w:w])
#     positions.append((0, w-tile_w))
#     tiles.append(img[h-tile_h:h, 0:tile_w])
#     positions.append((h-tile_h, 0))
#     tiles.append(img[h-tile_h:h, w-tile_w:w])
#     positions.append((h-tile_h, w-tile_w))

#     return tiles, positions

In [ ]:
# def process_data(image_dir, annotation_path, output_dir, tile_w=224, tile_h=224, min_area_ratio=0.1):
#     os.makedirs(output_dir, exist_ok=True)
#     df = pd.read_csv(annotation_path)
#     new_annotations = []

#     for filename in tqdm(df['filename'].unique(), desc=f"Tiling {os.path.basename(image_dir)}"):
#         img_path = os.path.join(image_dir, filename)
#         img = cv2.imread(img_path)
#         if img is None: continue

#         tiles, positions = crop_image_into_4tiles(img, tile_w, tile_h)
#         img_annotations = df[df['filename'] == filename]

#         for i, (tile, (start_y, start_x)) in enumerate(zip(tiles, positions)):
#             tile_filename = f"{os.path.splitext(filename)[0]}_tile_{i}.jpg"
#             tile_annotations = []

#             for _, row in img_annotations.iterrows():
#                 xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']
#                 new_xmin = max(0, xmin - start_x)
#                 new_ymin = max(0, ymin - start_y)
#                 new_xmax = min(tile_w, xmax - start_x)
#                 new_ymax = min(tile_h, ymax - start_y)

#                 if new_xmin < new_xmax and new_ymin < new_ymax:
#                     original_area = (xmax - xmin) * (ymax - ymin)
#                     new_area = (new_xmax - new_xmin) * (new_ymax - new_ymin)
#                     if new_area / original_area >= min_area_ratio:
#                         tile_annotations.append([tile_filename, tile_w, tile_h, row['class'], new_xmin, new_ymin, new_xmax, new_ymax])

#             if tile_annotations:
#                 cv2.imwrite(os.path.join(output_dir, tile_filename), tile)
#                 new_annotations.extend(tile_annotations)

#     new_df = pd.DataFrame(new_annotations, columns=['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax'])
#     new_df.to_csv(os.path.join(output_dir, "_annotations.csv"), index=False)
# # --- Run the Tiling ---
# process_data('./dataset_cropped_224x224_newest/train', './dataset_cropped_224x224_newest/train/_annotations.csv', './dataset_tiles/train')
# process_data('./dataset_cropped_224x224_newest/valid', './dataset_cropped_224x224_newest/valid/_annotations.csv', './dataset_tiles/valid')

# print("...Dataset tiling finished.")

Tiling train:   0%|          | 0/1869 [00:00<?, ?it/s]

Tiling valid:   0%|          | 0/647 [00:00<?, ?it/s]

...Dataset tiling finished.


In [ ]:
class TilingDataset(Dataset):
    def __init__(self, df, image_dir, transforms=None):
        self.df = df
        self.image_dir = image_dir
        self.transforms = transforms
        self.image_files = df['filename'].unique()
        self.class_to_label = {"Microplastic": 1}

    def __len__(self): return len(self.image_files)

    def __getitem__(self, idx):
        img_file = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_file)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape

        records = self.df[self.df['filename'] == img_file]
        boxes = records[['xmin', 'ymin', 'xmax', 'ymax']].values
        labels = [self.class_to_label[label] for label in records['class'].tolist()]

        #fixes the ValueError by ensuring no box exceeds image bounds
        boxes[:, [0, 2]] = boxes[:, [0, 2]].clip(0, w)
        boxes[:, [1, 3]] = boxes[:, [1, 3]].clip(0, h)

        #removin invalid boxes (where min >= max after clipping)
        keep = (boxes[:, 2] > boxes[:, 0]) & (boxes[:, 3] > boxes[:, 1])
        boxes = boxes[keep].tolist()
        labels = np.array(labels)[keep].tolist()


        if self.transforms:
            augmented = self.transforms(image=img, bboxes=boxes, class_labels=labels)
            img = augmented['image']
            boxes = augmented['bboxes']
            labels = augmented['class_labels']

        img_tensor = torch.from_numpy(img).permute(2, 0, 1).float() / 255.0
        boxes_tensor = torch.tensor(boxes, dtype=torch.float32)
        if boxes_tensor.shape[0] == 0: boxes_tensor = torch.zeros((0, 4), dtype=torch.float32)

        target = {'boxes': boxes_tensor, 'labels': torch.tensor(labels, dtype=torch.int64)}
        return img_tensor, target

transform = A.Compose([
    A.HorizontalFlip(p=0.5), A.VerticalFlip(p=0.2), A.Rotate(limit=30, p=0.5),
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

def collate_fn(batch): return tuple(zip(*batch))


## Model definition and Training/Evaluation

In [ ]:
def get_faster_rcnn_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=torchvision.models.detection.FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

def train_one_epoch(model, optimizer, data_loader, device):
    model.train()
    total_loss = 0
    for images, targets in tqdm(data_loader, desc="Training"):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        total_loss += losses.item()
    return total_loss / len(data_loader)

def evaluate(model, data_loader, device):
    metric = MeanAveragePrecision(box_format='xyxy').to(device)
    model.eval()
    with torch.no_grad():
        for images, targets in tqdm(data_loader, desc="mAP"):
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            predictions = model(images)
            metric.update(predictions, targets)
    return metric.compute()

def measure_inference_speed(model, device, num_images=100):
    print("Measuring inference speed...")
    model.eval()
    model.to(device)
    dummy_input = torch.randn(1, 3, 224, 224).to(device)
    for _ in range(10): _ = model(dummy_input) # Warmup

    start_time = time.time()
    with torch.no_grad():
        for _ in range(num_images):
            torch.cuda.synchronize()
            _ = model(dummy_input)
            torch.cuda.synchronize()
    end_time = time.time()

    avg_time_ms = (end_time - start_time) / num_images * 1000
    fps = 1.0 / ((end_time - start_time) / num_images)
    print(f"Average Inference Time: {avg_time_ms:.2f} ms")
    print(f"FPS: {fps:.2f}")
    return avg_time_ms, fps


## Execution

In [ ]:
if __name__ == "__main__":

    # --- Set Hyperparameters ---
    BATCH_SIZE = 8
    NUM_EPOCHS = 30
    NUM_WORKERS = 2
    LEARNING_RATE = 0.005

    # --- Setup Checkpoint Directory ---
    CHECKPOINT_DIR = "./checkpoints_tiled"
    os.makedirs(CHECKPOINT_DIR, exist_ok=True)
    os.makedirs("plots_tiled", exist_ok=True)

    # --- Setup Device ---
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print(f"Using device: {device}")
    NUM_CLASSES = 2

    # --- Load Data ---
    df_train = pd.read_csv("./content/dataset_cropped_224x224_newest/dataset_cropped_224x224_newest/train/_annotations.csv")
    df_valid = pd.read_csv("./content/dataset_cropped_224x224_newest/dataset_cropped_224x224_newest/valid/_annotations.csv")

    dataset_train = TilingDataset(df_train, "./content/dataset_cropped_224x224_newest/dataset_cropped_224x224_newest/train", transforms=transform)
    dataset_valid = TilingDataset(df_valid, "./content/dataset_cropped_224x224_newest/dataset_cropped_224x224_newest/valid", transforms=None)

    loader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, collate_fn=collate_fn)
    loader_valid = DataLoader(dataset_valid, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, collate_fn=collate_fn)

    # --- Initialize Model & Optimizer ---
    model = get_faster_rcnn_model(NUM_CLASSES)
    model.to(device)

    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=LEARNING_RATE, momentum=0.9, weight_decay=0.0005)

    # --- Count Parameters ---
    dummy_input = torch.randn(1, 3, 224, 224).to(device)
    macs, params = profile(model, inputs=(dummy_input, ), verbose=False)
    print(f"\nModel: ResNet-50 Faster R-CNN")
    print(f"Total Parameters: {params/1e6:.2f}M")

    # --- Start Training ---
    print("Starting training (Tiling + ResNet-50)...")
    train_start_time = time.time()
    history = []

    for epoch in range(NUM_EPOCHS):
        train_loss = train_one_epoch(model, optimizer, loader_train, device)
        mAP_results = evaluate(model, loader_valid, device)

        print(f"--- EPOCH {epoch+1}/{NUM_EPOCHS} ---")
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Valid mAP@0.5:    {mAP_results['map_50']:.4f}")
        print(f"Valid mAP@.5:.95: {mAP_results['map']:.4f}")

        # Save history
        history.append({
            "epoch": epoch + 1,
            "loss": train_loss,
            "mAP_50": float(mAP_results['map_50']),
            "mAP_50_95": float(mAP_results['map']),
        })

        # Save Checkpoint Locally (Fast)
        ckpt_path = os.path.join(CHECKPOINT_DIR, f"epoch_{epoch+1}.pth")
        torch.save(model.state_dict(), ckpt_path)
        print(f"Saved checkpoint to {ckpt_path}")

    train_end_time = time.time()
    total_train_time_min = (train_end_time - train_start_time) / 60

    print("...Training finished.")
    print(f"\n--- Total Training Time: {total_train_time_min:.2f} minutes ---")

    # --- Measure Speed ---
    print("\n--- Measuring Performance Metrics ---")
    measure_inference_speed(model, device)

    # --- Saving final model ---
    save_dir = "/models"
    os.makedirs(save_dir, exist_ok=True)
    final_drive_path = os.path.join(save_dir, "resnet50_tiled_final.pth")
    torch.save(model.state_dict(), final_drive_path)
    print(f"Saved final model to Google Drive: {final_drive_path}")


        # --- PLOT GRAPHS (Added back in) ---
    print("\nGenerating plots...")
    df_hist = pd.DataFrame(history)

    plt.figure()
    plt.plot(df_hist["epoch"], df_hist["loss"], marker="o")
    plt.title("Training Loss")
    plt.savefig(os.path.join(save_dir, "resnet_loss_graph.png"))
    plt.close()

    plt.figure()
    plt.plot(df_hist["epoch"], df_hist["mAP_50"], label="mAP50")
    plt.plot(df_hist["epoch"], df_hist["mAP_50_95"], label="mAP50:95")
    plt.legend()
    plt.title("Validation mAP")
    plt.savefig(os.path.join(save_dir, "resnet_map_graph.png"))
    plt.close()
    print("Graphs saved to Google Drive.")


Using device: cuda
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


100%|██████████| 160M/160M [00:01<00:00, 124MB/s]



Model: ResNet-50 Faster R-CNN
Total Parameters: 41.30M
Starting training (Tiling + ResNet-50)...


Training:   0%|          | 0/234 [00:00<?, ?it/s]

mAP:   0%|          | 0/81 [00:00<?, ?it/s]

...

mAP:   0%|          | 0/81 [00:00<?, ?it/s]

--- EPOCH 29/30 ---
Train Loss: 0.3181
Valid mAP@0.5:    0.7287
Valid mAP@.5:.95: 0.3346
Saved checkpoint to ./checkpoints_tiled/epoch_29.pth


Training:   0%|          | 0/234 [00:00<?, ?it/s]

mAP:   0%|          | 0/81 [00:00<?, ?it/s]

--- EPOCH 30/30 ---
Train Loss: 0.3217
Valid mAP@0.5:    0.7281
Valid mAP@.5:.95: 0.3349
Saved checkpoint to ./checkpoints_tiled/epoch_30.pth
...Training finished.

--- Total Training Time: 219.45 minutes ---

--- Measuring Performance Metrics ---
Measuring inference speed...
Average Inference Time: 86.43 ms
FPS: 11.57
✅ Saved final model to Google Drive: /content/drive/MyDrive/DL_resnet_results/resnet50_tiled_final.pth

Generating plots...
✅ Graphs saved to Google Drive.
